<a href="https://colab.research.google.com/github/ArindamBanerji/wip-experiments/blob/master/misc/TS_Image_architectures_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00


In [2]:
!pip install yfinance

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
from torch.autograd import Variable
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader,ConcatDataset, TensorDataset,Subset,Dataset
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
import yfinance as yf
import datetime
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField as GADF
import plotly.graph_objs as go
import plotly.io as pio
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from pyts.image import GramianAngularField
from torchvision.models import resnet18
import torch.optim as optim

In [4]:
warnings.filterwarnings('ignore')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


**Import DataSets**

In [5]:
# create function that takes ticker symbol, start and end daates and returns dataset
def get_data(ticker, start_date, end_date):
  data = yf.download(ticker, start=start_date, end=end_date, interval='1h')
  return data

In [6]:
# define function to show sample of returned data
def show_sample(data):
  print(data.head())
  print(data.tail())
  print(data.describe())
  print(data.info())

In [7]:
start_date = '2023-05-11'
end_date = '2024-06-10'
ticker = 'AAPL'
data = get_data(ticker, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [8]:
# shiw sample data
show_sample(data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-05-11 09:30:00-04:00  173.850006  173.960007  172.169998  172.669998   
2023-05-11 10:30:00-04:00  172.669998  173.699997  172.300003  173.699997   
2023-05-11 11:30:00-04:00  173.695007  173.914993  173.259995  173.660004   
2023-05-11 12:30:00-04:00  173.669998  174.024994  173.380005  173.470001   
2023-05-11 13:30:00-04:00  173.470001  174.000000  173.059998  173.990097   

                            Adj Close    Volume  
Datetime                                         
2023-05-11 09:30:00-04:00  172.669998  11552802  
2023-05-11 10:30:00-04:00  173.699997   6827208  
2023-05-11 11:30:00-04:00  173.660004   5325734  
2023-05-11 12:30:00-04:00  173.470001   4415832  
2023-05-11 13:30:00-04:00  173.990097   4089255  
                                 Open        High         Low       Close  \
Datetime                         

**Create GADF images from Time Series**

In [ ]:
def create_gaf_images(data, image_size=28):
    num_days = len(data) // (image_size)
    data = data[:num_days*image_size].reshape(num_days, image_size)
    close_prices=[]
    for i in range(num_days):
        close_prices.append(np.mean(data[i]))
    gadf = GADF(image_size=image_size)
    gadf_images= gadf.fit_transform(data)
    return gadf_images,close_prices


In [ ]:
# def function to return np.array of a particular column of dataframe
def get_column(data, column_name):
    return np.array(data[column_name])

In [ ]:
def get_for_whole(data):
    images=[]
    prices=[]
    data1= data.drop(columns=['Volume'])
    for idx, i in enumerate(data1.columns):
      if(idx<1 or idx>3):
        continue
      close= np.array(data1[i])
      gaf_images,close_prices= create_gaf_images(close,30)
      images.append(gaf_images)
      if(i=='Close'):
        prices.append(close_prices)
    images= np.transpose(images, (1, 0, 2, 3))
    return np.array(images), prices


In [ ]:
images, prices= get_for_whole(data)
# get 'Close' data and print len
close = get_column(data, 'Close')
print(len(close))

In [ ]:
# define function to show an image at a specified index
def show_image(images, index):
  image= images[index]
  image= np.transpose(image, (1, 2, 0))
  plt.imshow(image)

In [ ]:
gaf_images,close_prices= create_gaf_images(close,30)
plt.imshow(gaf_images[0],cmap='hot')
print(gaf_images.shape)

In [ ]:
show_image(images, 0)

** Generate Encodings usng CNNs on GADF images**

In [ ]:
# This function is used to generate random noise image
def generate_noise_image():
    return torch.rand((1, 3, 224, 224))

# function to preprocess input image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# function to get embeddings from ResNet model
def get_resnet_embeddings(image_path=None):

    resnet = models.resnet50(pretrained=True)
    resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
    if image_path is None:
        image = generate_noise_image()
    else:
        image = preprocess_image(image_path)
    embeddings = resnet(image)
    return embeddings

# function to get embeddings from DenseNet model
def get_densenet_embeddings(image_path=None):
    densenet = models.densenet121(pretrained=True)
    densenet = torch.nn.Sequential(*list(densenet.children())[:-1])
    if image_path is None:
        image = generate_noise_image()
    else:
        image = preprocess_image(image_path)
    embeddings = densenet(image)
    return embeddings

**Simple LSTM**

In [9]:
class LstmNet(nn.Module):
    def __init__(self,input_size,hidden_size,layers,output_size):
        super(LstmNet,self).__init__()
        self.layers = layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=layers, batch_first=True)

        self.fc1 = nn.Linear(hidden_size, output_size)
        # self.fc2 = nn.Linear(10,output_size)
    def forward(self,x):
        # print(x.shape)
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_size)).to(device)

        c0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_size)).to(device)
        out, (h_out, c_out) = self.lstm(x,(h0,c0))
        # print(out.shape,h_out.shape,c_out.shape)
        out = self.fc1(out[:,-1,:])
        return out.squeeze(1)


In [10]:
def prepare_dataset(data,length):
    x = []
    y = []
    for i in range(len(data)-length-1):
        x.append(data[i:i+length])
        y.append(data[i+length])
    return np.array(x),np.array(y)

def train_test_split(X,Y,percent):
    per = percent/100
    sz = len(X)
    xtrain = torch.Tensor(X[:int(sz*per)])
    ytrain = torch.Tensor(Y[:int(sz*per)])
    xtest = torch.Tensor(X[int(sz*per):])
    ytest = torch.Tensor(Y[int(sz*per):])
    return xtrain,ytrain,xtest,ytest

class DataPrep(Dataset):
  def __init__(self, inputs, targets):
      self.inputs = inputs
      self.targets = targets
  def __len__(self):
      return len(self.inputs)

  def __getitem__(self, index):
      X = self.inputs[index]
      Y = self.targets[index]
      return X, Y

In [11]:
# define function to encapsulate the whole train-prediction cycle

def train_predict (data) :
    scaler = StandardScaler()
    values = scaler.fit_transform(data['Close'].values.reshape(-1,1))
    seq_len = 40
    data_inp,data_tar = prepare_dataset(values,seq_len)
    xtrain,ytrain,xtest,ytest = train_test_split(data_inp,data_tar,80)

    traindata = DataPrep(xtrain,ytrain)
    testdata = DataPrep(xtest,ytest)
    batch_size = 32
    trainset = DataLoader(traindata,batch_size = batch_size,shuffle = True)
    testset = DataLoader(testdata,batch_size = batch_size,shuffle = True)
    for xbatch,ybatch in trainset:
        print(xbatch.shape,ybatch.shape)
        break


    train_losses = []
    test_losses = []
    input_sz = 1
    hidden_sz = 200
    output_sz = 1
    layers = 2
    model = LstmNet(input_sz,hidden_sz,layers,output_sz).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)
    for epoch in range(1000):
        batch_loss = 0
        # correct = 0
        for xbatch,ybatch in trainset:
            xbatch,ybatch = xbatch.to(device),ybatch.to(device)
            out = model(xbatch)
            # print(ybatch.shape,out.shape)
            loss = criterion(out, ybatch.squeeze(1))
            loss.backward()
            optimizer.step()
            batch_loss += loss.item()
            # break

        train_loss = batch_loss/len(trainset)
        batch_loss = 0
        with torch.no_grad():
            for xbatch,ybatch in testset:
                xbatch,ybatch = xbatch.to(device),ybatch.to(device)
                out = model(xbatch)
                loss = criterion(out, ybatch.squeeze(1))
                batch_loss += loss.item()
        test_loss = batch_loss/len(testset)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        if(epoch%10==9):
            print("Epoch: ", epoch+1, "|", "Train Loss : ", "{:.6f}".format(train_loss), "|", "Test Loss : ", "{:.6f}".format(test_loss))

    return xtrain, ytrain, xtest, ytest, model



In [12]:
# define the prdict function for lstm
def predict(xtrain, ytrain, xtest, ytest, model):
  train_pred = model(xtrain.to(device)).cpu().data.numpy()
  train_actual = ytrain.data.numpy()
  test_pred = model(xtest.to(device)).cpu().data.numpy()
  test_actual = ytest.data.numpy()

  pred = np.concatenate((train_pred,test_pred))
  # pred = scaler.inverse_transform(pred)
  actual = np.concatenate((train_actual,test_actual)).squeeze()
  # actual = scaler.inverse_transform(pred)
  print(pred.shape)
  print(actual.shape)

  return pred, actual, train_pred, test_actual

In [13]:
# define function to display results
def display_results(pred, actual, train_pred, test_actual) :
    trace1 = go.Scatter(x = [(i+1) for i in range(len(pred))],y = pred, name='Predicted Data')
    trace2 = go.Scatter(x = [(i+1) for i in range(len(actual))],y = actual, name='Actual data')

    datas = [trace1,trace2]

    layout = go.Layout(title='Prediction for 80:20 split Normal LSTM')
    fig = go.Figure(data=datas, layout=layout)
    fig.add_vline(x=len(train_pred), line_width=1, line_dash="dash", line_color="red")

    fig.show()



In [14]:
def run_lstm (data) :
  xtrain, ytrain, xtest, ytest, model = train_predict(data)
  pred, actual, train_pred, test_actual = predict(xtrain, ytrain, xtest, ytest, model)
  display_results(pred, actual, train_pred, test_actual)


In [15]:
run_lstm(data)

torch.Size([32, 40, 1]) torch.Size([32, 1])
Epoch:  10 | Train Loss :  0.787392 | Test Loss :  1.597036
Epoch:  20 | Train Loss :  0.333614 | Test Loss :  0.531240
Epoch:  30 | Train Loss :  0.168657 | Test Loss :  0.102958
Epoch:  40 | Train Loss :  0.485729 | Test Loss :  0.981923
Epoch:  50 | Train Loss :  0.034085 | Test Loss :  0.089765
Epoch:  60 | Train Loss :  0.021104 | Test Loss :  0.034387
Epoch:  70 | Train Loss :  0.288492 | Test Loss :  0.526770
Epoch:  80 | Train Loss :  0.051126 | Test Loss :  0.079130
Epoch:  90 | Train Loss :  0.062237 | Test Loss :  0.066283
Epoch:  100 | Train Loss :  0.168391 | Test Loss :  0.356529
Epoch:  110 | Train Loss :  0.057050 | Test Loss :  0.101631
Epoch:  120 | Train Loss :  0.110871 | Test Loss :  0.190657
Epoch:  130 | Train Loss :  0.066445 | Test Loss :  0.062442
Epoch:  140 | Train Loss :  0.101405 | Test Loss :  0.157043
Epoch:  150 | Train Loss :  0.161023 | Test Loss :  0.308615
Epoch:  160 | Train Loss :  0.055633 | Test Loss :